In [9]:
# #importing openslide:
# OPENSLIDE_PATH = r'C:\Users\Kevin\Desktop\openslide-win64-20221217\bin'
#
# import os
#
# if hasattr(os, 'add_dll_directory'):
#     # Python >= 3.8 on Windows
#     with os.add_dll_directory(OPENSLIDE_PATH):
#         import openslide
# else:
#     import openslide

In [1]:
import os
add_dll_dir = getattr(os, "add_dll_directory", None)
vipsbin = r'C:\Users\Kevin\Downloads\vips-dev-w64-web-8.14.1-static.zip\vips-dev-8.14\bin'
# vipsbin = 'c:\vips-dev-8.14\bin' # LibVIPS binary dir
if callable(add_dll_dir):
    add_dll_dir(vipsbin)
else:
    os.environ["PATH"] = os.pathsep.join((vipsbin, os.environ["PATH"]))

import pyvips

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Kevin\\Downloads\\vips-dev-w64-web-8.14.1-static.zip\\vips-dev-8.14\\bin'

In [5]:
import os
vipshome = r'C:\Users\Kevin\Downloads\vips-dev-w64-web-8.14.1-static.zip\vips-dev-8.14\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips

OSError: cannot load library 'libgobject-2.0-0.dll': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0.dll'

In [2]:
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import skimage.measure
import scipy.stats as stats
from time import time
from skimage.measure import label, regionprops_table
import pyvips

OSError: cannot load library 'libgobject-2.0-0.dll': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libgobject-2.0-0.dll'

In [7]:
#stitch tiles into wsi
def stitch(tiles,img_height,img_width,img_height2,img_width2,channels):
    wsi = tiles.swapaxes(1,2)
    wsi = wsi.reshape(img_height2,img_width2,channels) #tiles are padded, so use padded image size to stitch
    wsi = wsi[:img_height,:img_width,:] #remove pad
    return np.squeeze(wsi)

In [4]:
# src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
# imnm = 'OTS_14684_3.ndpi'
# imobj = openslide.open_slide(os.path.join(src,imnm))
# imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])
# imobj.level_dimensions[1]

In [5]:
#wsi to tiles
def reshape_split(image:np.ndarray,kernel_size:tuple):
    img_height,img_width,channels=image.shape
    tile_height,tile_width = kernel_size
    tiled_array = image.reshape(img_height//tile_height,
                                tile_height,
                                img_width//tile_width,
                                tile_width,
                                channels)
    tiled_array = tiled_array.swapaxes(1,2)
    return tiled_array

In [85]:
src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
imnm = 'OTS_14832_3.ndpi'
imobj = openslide.open_slide(os.path.join(src,imnm))
imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])

# Image to Array
imnp = np.array(imobj)
imobj.close()
h,w,_=imnp.shape
tile_height, tile_width = (1024,1024)
# Padding (so that it's in multiples of 1024)
imnpr = np.pad(imnp, pad_width=[(0, tile_height-h%tile_height),(0, tile_width-w%tile_width),(0, 0)], mode='constant', constant_values=0)
# imnpr = imnpr / 127.5 - 1 #normalize [-1 1]
# imnpr = imnpr / 255 #normalize [0 1]
img_height2,img_width2,channels=imnpr.shape
# Tile
tiles = reshape_split(imnpr, (1024,1024))

In [86]:
img_width2

113664

In [87]:
print(tiles.shape)
nrows = tiles.shape[0]
ncols = tiles.shape[1]

(54, 111, 1024, 1024, 4)


In [88]:
imnp.shape

(54912, 113280, 4)

In [89]:
imnpr.shape

(55296, 113664, 4)

In [15]:
# save_src = r'\\shelter\Kyu\unstain2stain\tiles\not_registrated\Unstained'
# for idx in range(0,tiles.shape[0]):
#     for idx1 in range(0,tiles.shape[1]):
#         tmp_tile = tiles[idx][idx1]
#         save_path = os.path.join(save_src,str(idx)+str(idx1)+'tile'+'.png')
#         Image.fromarray(tmp_tile).save(save_path)

In [16]:
src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
imnm = 'OTS_14832_3_he.ndpi'
imobj = openslide.open_slide(os.path.join(src,imnm))
imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])

# Image to Array
imnp = np.array(imobj)
imobj.close()
h,w,_=imnp.shape
tile_height, tile_width = (1024,1024)
# Padding
imnpr = np.pad(imnp, pad_width=[(0, tile_height-h%tile_height),(0, tile_width-w%tile_width),(0, 0)], mode='constant', constant_values=0)
# imnpr = imnpr / 127.5 - 1 #normalize [-1 1]
# imnpr = imnpr / 255 #normalize [0 1]
img_height2,img_width2,channels=imnpr.shape
# Tile
tiles = reshape_split(imnpr, (1024,1024))

In [18]:
# save_src = r'\\shelter\Kyu\unstain2stain\tiles\not_registrated\HE'
# for idx in range(0,tiles.shape[0]):
#     for idx1 in range(0,tiles.shape[1]):
#         tmp_tile = tiles[idx][idx1]
#         save_path = os.path.join(save_src,str(idx)+str(idx1)+'tile'+'.png')
#         Image.fromarray(tmp_tile).save(save_path)

### The tile generation code above is unregistered, I ran a code in matlab to generate the registered tiles, which have almost the same # of tiles, except that there are some missing images. Find the difference:

In [90]:
stain_tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE\OTS_14832_3_he'
unstain_tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained\OTS_14832_3'

In [91]:
unstain_tile_name = os.listdir(unstain_tile_src)
stain_tile_name = os.listdir(stain_tile_src)

In [92]:
print(len(stain_tile_name))
print(len(unstain_tile_name))

10996
10996


In [93]:
unstain_tile_base = [os.path.basename(x) for x in unstain_tile_name]
stain_tile_base = [os.path.basename(x) for x in stain_tile_name]
unstain_list = [ x[-10:-4] for x in unstain_tile_base]
stain_list = [ x[-10:-4] for x in stain_tile_base]

In [94]:
not_in_stain = [x for x in unstain_list if x not in stain_list]
not_in_stain

[]

### Found the three differences- delete them from unstained, and run inference on pix2pix project. After inference, stitch back image:

In [95]:
inferred_src = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_pix2pix\test_pix2pix_latest\images'
inferred_list = os.listdir(inferred_src)
inferred_name = [os.path.basename(x) for x in inferred_list]
real_inferred2 = [x for x in inferred_name if x[-10:-4]  == 'real_B']
real_inferred_src = [os.path.join(inferred_src,x) for x in real_inferred2]
print(len(real_inferred_src)) #10995

10995


In [3]:
real_inferred_src[0]

10995


In [4]:
import shutil
destination = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_pix2pix\test_pix2pix_latest\images\realB'

for idx in real_inferred_src:
    shutil.copy(idx, destination)

'C:\\Users\\Kevin\\PycharmProjects\\pix2pix\\pytorch-CycleGAN-and-pix2pix\\results\\unstain2stain_pix2pix\\test_pix2pix_latest\\images\\100482_13135xy4369_fake_B.png'

In [6]:
import os
import shutil

src_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_pix2pix\test_pix2pix_latest\images'
# Destination directory to copy the images
dst_dir = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_pix2pix\test_pix2pix_latest\images\realB'


# Iterate over all files in the source directory
for file_name in os.listdir(src_dir):

    # Check if the file is an image file
    if file_name.endswith(".png") and file_name[-10:-4]  == 'real_B':

        # Create the full path of the source file
        src_file = os.path.join(src_dir, file_name)

        # Create a unique destination file name by appending a timestamp to the original file name
        timestamp = str(int(os.path.getmtime(src_file)))
        dst_file = os.path.join(dst_dir, file_name[:-4] + '_' + timestamp + '.png')

        # Copy the file from the source directory to the destination directory
        shutil.copyfile(src_file, dst_file)

print("All images copied successfully!")


In [4]:
# Set the directory containing the image tiles
directory = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_pix2pix\test_pix2pix_latest\images\realB'

# Set the size of the final image (size of Unstained image OTS_14832_3)
x1 = 200000
y1 = 100000

# Create a blank, black image with the specified size
final_image = Image.new('RGB', (x1, y1), color='black')

# Loop over all of the image tiles in the directory
for filename in os.listdir(directory):
    if filename.endswith('.png'):
        # Extract the x and y coordinates from the filename
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))

        # Open the image tile and paste it onto the final image at the specified coordinates
        tile_image = Image.open(os.path.join(directory, filename))
        final_image.paste(tile_image, (x, y))

im = pyvips.Image.new_from_buffer(final_image)
image_height = im.height
image_bands = im.bands
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")
src = r'\\shelter\Kyu\unstain2stain'
im.tiffsave(os.path.join(src, 'final_image.ome.tiff'), compression="jpeg", tile=True, tile_width=512,
            tile_height=512, pyramid=True, subifd=True)

NameError: name 'pyvips' is not defined

In [ ]:

src = r'\\shelter\Kyu\unstain2stain'
ims = [_ for _ in os.listdir(src) if _.endswith('tif')]
im1 = pyvips.Image.new_from_file(os.path.join(src, ims[0]))
im = im1
image_height = im.height
image_bands = im.bands

# split to separate image planes and stack vertically ready for OME
# im = pyvips.Image.arrayjoin(im.bandsplit(), across=1)
# set minimal OME metadata
# before we can modify an image (set metadata in this case), we must take a
# private copy
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")

im.tiffsave(os.path.join(src, 'multi_channel_pyramid.ome.tiff'), compression="jpeg", tile=True, tile_width=512,
            tile_height=512, pyramid=True, subifd=True)

In [3]:
from PIL import Image

with Image.open(r'\\shelter\Kyu\unstain2stain\test_final_image.png') as img:
    # do something with the image, such as display or process it
    img.show()


In [ ]:
import sys
import pyvips

im = pyvips.Image.new_from_file('/Users/chamisulhuresh/Desktop/test.ndpi')

# openslide will add an alpha ... drop it
if im.hasalpha():
    im = im[:-1]

image_height = im.height
image_bands = im.bands

# split to separate image planes and stack vertically ready for OME
im = pyvips.Image.arrayjoin(im.bandsplit(), across=1)

# set minimal OME metadata
# before we can modify an image (set metadata in this case), we must take a
# private copy
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")

im.tiffsave('/Users/chamisulhuresh/Desktop/ndpitest.ome.tiff', compression="jpeg", tile=True,
            tile_width=512, tile_height=512,
            pyramid=True, subifd=True)

In [3]:
img = Image.open(r'\\shelter\Kyu\unstain2stain\test_final_image.png')
imgra = np.array(img)
imgra2 = cv2.resize(imgra,(imgra.shape[0]//1000, imgra.shape[1]//1000), interpolation=cv2.INTER_AREA)
imgra2.shape
# Image.fromarray(imgra2).show()

MemoryError: 

### Try registering the two tiles with optical flow to compare with matlab rigid registration:


In [11]:
def registrate_two_images(reference_image_path, moving_image_path, save_path):
    """
    Note: The order of the files saved in the ref_img_path and mov_img_path must be the same so that you are registering the same images!
    """
    ref_img_path = [_ for _ in os.listdir(reference_image_path) if _.endswith(".png")]
    ref_img_path_complete = [os.path.join(reference_image_path, x) for x in ref_img_path]
    mov_img_path = [_ for _ in os.listdir(moving_image_path) if _.endswith(".png")]
    mov_img_path_complete = [os.path.join(moving_image_path, x) for x in mov_img_path]
    mov_img_name = [x.replace('.png','') for x in mov_img_path]
    if int(len(ref_img_path)) != int(len(mov_img_path)):
        print("Number of images in reference and moving file paths are not equal, please fix and try again!")
        return

    start = time()
    for idx in range(0,len(ref_img_path_complete)):
        ref_img = Image.open(ref_img_path_complete[idx])
        mov_img = Image.open(mov_img_path_complete[idx])
        ref_img = np.array(ref_img)
        mov_img = np.array(mov_img)
        ref_img_g = cv2.cvtColor(ref_img,cv2.COLOR_RGBA2GRAY)
        mov_img_g = cv2.cvtColor(mov_img,cv2.COLOR_RGBA2GRAY)
        v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
        nr, nc = ref_img_g.shape
        row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                             indexing='ij')
        mov_img_warp_ra =[]
        for i in range(3):
            mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
            mov_img_warp_ra.append(mov_img_warp)
        r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
        g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
        b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
        rgb = np.stack([r,g,b],axis=2)
        reg_img = Image.fromarray(rgb)
        print(idx)
        reg_img.save(os.path.join(save_path,str(mov_img_name[idx]) + '.png'))

    end = time()
    print("time it took to register: "+  str((end-start)/60) + " minutes")


### Possible dummy code to recognize H&E images and discard images that are not H&E:

In [ ]:
import cv2
import os

# Set path to the folder containing the images
image_folder = 'path/to/folder'

# Set threshold for H&E stained images (adjust as needed)
threshold = 0.1

# Loop through each image in the folder
for image_file in os.listdir(image_folder):
    # Read the image
    image = cv2.imread(os.path.join(image_folder, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate the blue-green ratio (BGR) using the H&E staining method
    blue = image[:, :, 0].astype(float)
    green = image[:, :, 1].astype(float)
    red = image[:, :, 2].astype(float)
    bgr = (blue - (red + green) / 2) / (blue + green + red)
    bgr_mean = bgr.mean()

    # If the BGR ratio is below the threshold, delete the image
    if bgr_mean < threshold:
        os.remove(os.path.join(image_folder, image_file))


In [32]:
import os
import cv2

# Define the directory where the image tiles are stored
img_dir = r'"\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\HE\OTS_14684_3_he - Copy'

# Define the threshold value for identifying empty tiles
threshold_value = 200
removed_name = []
# Loop through all the image files in the directory
for filename in os.listdir(img_dir):
    # Load the image
    img = cv2.imread(os.path.join(img_dir, filename))

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Calculate the mean pixel value of the grayscale image
    mean_pixel_value = cv2.mean(gray)[0]

    # If the mean pixel value is below the threshold, assume the tile is empty and discard it
    if mean_pixel_value < threshold_value:
        os.remove(os.path.join(img_dir, filename))
        print('Removed:', filename)
        removed_name.append(filename)
    else:
        print('Kept:', filename)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_24640\2577889262.py:22: RuntimeWarning: invalid value encountered in divide
  bgr = (blue - (red + green) / 2) / (blue + green + red)


54


In [34]:
### Code to drop out low MI tiles:

['1126_1126xy0001.png',
 '1126_8294xy0008.png',
 '52326_73830xy3972.png',
 '53350_18534xy3996.png',
 '53350_25702xy4003.png',
 '53350_27750xy4005.png',
 '53350_32870xy4010.png',
 '53350_33894xy4011.png',
 '53350_37990xy4015.png',
 '53350_40038xy4017.png',
 '53350_41062xy4018.png',
 '53350_42086xy4019.png',
 '53350_47206xy4024.png',
 '53350_48230xy4025.png',
 '53350_61542xy4038.png',
 '53350_62566xy4039.png',
 '53350_63590xy4040.png',
 '53350_64614xy4041.png',
 '53350_65638xy4042.png',
 '53350_66662xy4043.png',
 '54374_26726xy4082.png',
 '54374_27750xy4083.png',
 '54374_32870xy4088.png',
 '54374_33894xy4089.png',
 '54374_36966xy4092.png',
 '54374_39014xy4094.png',
 '54374_40038xy4095.png',
 '54374_41062xy4096.png',
 '54374_43110xy4098.png',
 '54374_62566xy4117.png',
 '54374_63590xy4118.png',
 '54374_64614xy4119.png',
 '54374_65638xy4120.png',
 '55398_23654xy4157.png',
 '55398_24678xy4158.png',
 '55398_25702xy4159.png',
 '55398_26726xy4160.png',
 '55398_27750xy4161.png',
 '55398_28774xy4